In [1]:
%load_ext autotime
%matplotlib inline

In [42]:
import numpy as np
from numpy.random import choice, permutation, rand
from numpy import random
from sklearn.preprocessing import MinMaxScaler

time: 418 µs


In [8]:
# https://github.com/1313e/e13Tools/blob/master/e13tools/sampling/lhs.py

def lhd(n_sam, n_val, val_rng=None, method='random', criterion=None,
        iterations=1000, get_score=False, quickscan=True, constraints=None):
    """
    Generates a Latin Hypercube Design of `n_sam` samples, each with `n_val`
    values. Method for choosing the 'best' Latin Hypercube Design depends on
    the `method` and `criterion` that are used.
    Parameters
    ----------
    n_sam : int
        The number of samples to generate.
    n_val : int
        The number of values in a single sample.
    Optional
    --------
    val_rng : 2D array_like or None. Default: None
        Array defining the lower and upper limits of every value in a sample.
        Requires: numpy.shape(val_rng) = (`n_val`, 2).
        If *None*, output is normalized.
    method : {'random'; 'fixed'; 'center'}. Default: 'random'
        String specifying the method used to construct the Latin Hypercube
        Design. See ``Notes`` for more details.
        If `n_sam` == 1 or `n_val` == 1, `method` is set to the closest
        corresponding method if necessary.
    criterion : float, {'maximin'; 'correlation'; 'multi'} or None. \
        Default: None
        Float or string specifying the criterion the Latin Hypercube Design has
        to satisfy or *None* for no criterion. See ``Notes`` for more details.
        If `n_sam` == 1 or `n_val` == 1, `criterion` is set to the closest
        corresponding criterion if necessary.
    iterations : int. Default: 1000
        Number of iterations used for the criterion algorithm.
    get_score : bool. Default: False
        If *True*, the normalized maximin, correlation and multi scores are
        also returned if a criterion is used.
    quickscan : bool. Default: True
        If *True*, a faster but less precise algorithm will be used for the
        criteria.
    constraints : 2D array_like or None. Default: None
        If `constraints` is not empty and `criterion` is not *None*, `sam_set`
        + `constraints` will satisfy the given criterion instead of `sam_set`.
        Providing this argument when `criterion` is *None* will discard it.
        **WARNING**: If `constraints` is not a 'fixed' or 'center' lay-out LHD,
        the output might contain errors.
    Returns
    -------
    sam_set : 2D :obj:`~numpy.ndarray` object
        Sample set array of shape [`n_sam`, `n_val`].
    Notes
    -----
    The 'method' argument specifies the way in which the values should be
    distributed within the value intervals.
    The following methods can be used:
    ======== ===================================
    method   interval lay-out
    ======== ===================================
    'random' Values are randomized
    'fixed'  Values are fixed to maximize spread
    'center' Values are centered
    'r'      Same as 'random'
    'f'      Same as 'fixed'
    'c'      Same as 'center'
    ======== ===================================
    The 'fixed' method chooses values in such a way, that the distance between
    the values is maxed out.
    The 'criterion' argument specifies how much priority should be given to
    maximizing the minimum distance and minimizing the correlation between
    samples. Strings specify basic priority cases, while a value between 0 and
    1 specifies a custom case.
    The following criteria can be used (last column shows the equivalent
    float value):
    ============= ==================================================== ======
    criterion     effect/priority                                      equiv
    ============= ==================================================== ======
    None          No priority                                          --
    'maximin'     Maximum priority for maximizing the minimum distance 0.0
    'correlation' Maximum priority for minimizing the correlation      1.0
    'multi'       Equal priority for both                              0.5
    [0, 1]        Priority is given according to value provided        --
    ============= ==================================================== ======
    Examples
    --------
    Latin Hypercube with 5 samples with each 2 random, fixed or centered
    values:
        >>> import numpy as np
        >>> np.random.seed(0)
        >>> lhd(5, 2, method='random')
        array([[ 0.34303787,  0.55834501],
               [ 0.70897664,  0.70577898],
               [ 0.88473096,  0.19273255],
               [ 0.1097627 ,  0.91360891],
               [ 0.52055268,  0.2766883 ]])
        >>> lhd(5, 2, method='fixed')
        array([[ 0.5 ,  0.75],
               [ 0.25,  0.25],
               [ 0.  ,  1.  ],
               [ 0.75,  0.5 ],
               [ 1.  ,  0.  ]])
        >>> lhd(5, 2, method='center')
        array([[ 0.1,  0.9],
               [ 0.9,  0.5],
               [ 0.5,  0.7],
               [ 0.3,  0.3],
               [ 0.7,  0.1]])
    Latin Hypercube with 4 samples, 3 values in a specified value range:
        >>> import numpy as np
        >>> np.random.seed(0)
        >>> val_rng = [[0, 2], [1, 4], [0.3, 0.5]]
        >>> lhd(4, 3, val_rng=val_rng)
        array([[ 1.30138169,  2.41882975,  0.41686981],
               [ 0.27440675,  1.32819041,  0.48240859],
               [ 1.77244159,  3.53758114,  0.39180394],
               [ 0.85759468,  3.22274707,  0.31963924]])
    Latin Hypercubes can also be created by specifying a criterion with either
    a string or a normalized float. The strings identify basic float values.
        >>> import numpy as np
        >>> np.random.seed(0)
        >>> lhd(4, 3, method='fixed', criterion=0)
        array([[ 0.66666667,  0.        ,  0.66666667],
               [ 1.        ,  0.66666667,  0.        ],
               [ 0.33333333,  1.        ,  1.        ],
               [ 0.        ,  0.33333333,  0.33333333]])
        >>> np.random.seed(0)
        >>> lhd(4, 3, method='fixed', criterion='maximin')
        array([[ 0.66666667,  0.        ,  0.66666667],
               [ 1.        ,  0.66666667,  0.        ],
               [ 0.33333333,  1.        ,  1.        ],
               [ 0.        ,  0.33333333,  0.33333333]])
    """

    # Make sure that if val_rng is given, that it is valid
    if val_rng is not None:
        # If val_rng is 1D, convert it to 2D (expected for 'n_val' = 1)
        val_rng = np.array(val_rng, ndmin=2)

        # Check if the given val_rng is in the correct shape
        if not(val_rng.shape == (n_val, 2)):
            raise ShapeError("'val_rng' has incompatible shape: %s != (%s, %s)"
                             % (val_rng.shape, n_val, 2))

    # TODO: Implement constraints method again!
    # Make sure that constraints is a numpy array
    if constraints is not None:
        constraints = np.array(constraints, ndmin=2)

    if constraints is None:
        pass
    elif(constraints.shape[-1] == 0):
        # If constraints is empty, there are no constraints
        constraints = None
    elif(constraints.ndim != 2):
        # If constraints is not two-dimensional, it is invalid
        raise ShapeError("Constraints must be two-dimensional!")
    elif(constraints.shape[-1] == n_val):
        # If constraints has the same number of values, it is valid
        constraints = _extract_sam_set(constraints, val_rng)
    else:
        # If not empty and not right shape, it is invalid
        raise ShapeError("Constraints has incompatible number of values: "
                         "%s =! %s" % (np.shape(constraints)[1], n_val))

    # Check for cases in which some methods make no sense
    if(n_sam == 1 and method.lower() in ('fixed', 'f')):
        method = 'center'
    elif(criterion is not None and method.lower() in ('random', 'r')):
        method = 'fixed'

    # Check for cases in which some criterions make no sense
    # If so, criterion will be changed to something useful
    if criterion is None:
        pass
    elif(n_sam == 1):
        criterion = None
    elif(n_val == 1 or n_sam == 2):
        criterion = None
    elif isinstance(criterion, (int, float)):
        if not(0 <= criterion <= 1):
            raise ValueError("Input argument 'criterion' can only have a "
                             "normalized value as a float value!")
    elif criterion.lower() not in ('maximin', 'correlation', 'multi'):
        raise ValueError("Input argument 'criterion' can only have {'maximin',"
                         " 'correlation', 'multi'} as string values!")

    # Pick correct lhs-method according to method
    if method.lower() in ('random', 'r'):
        sam_set = _lhd_random(n_sam, n_val)
    elif method.lower() in ('fixed', 'f'):
        sam_set = _lhd_fixed(n_sam, n_val)
    elif method.lower() in ('center', 'c'):
        sam_set = _lhd_center(n_sam, n_val)

    # Pick correct criterion
    if criterion is not None:
        multi_obj = Multi_LHD(sam_set, criterion, iterations, quickscan,
                              constraints)
        sam_set, mm_val, corr_val, multi_val = multi_obj()

    # If a val_rng was given, scale sam_set to this range
    if val_rng is not None:
        # Scale sam_set according to val_rng
        sam_set = val_rng[:, 0]+sam_set*(val_rng[:, 1]-val_rng[:, 0])

    if get_score and criterion is not None:
        return(sam_set, np.array([mm_val, corr_val, multi_val]))
    else:
        return(sam_set)

    
def _lhd_random(n_sam, n_val):
    # Generate the equally spaced intervals/bins
    bins = np.linspace(0, 1, n_sam+1)

    # Obtain lower and upper bounds of bins
    bins_low = bins[0:n_sam]
    bins_high = bins[1:n_sam+1]

    # Pair values randomly together to obtain random samples
    sam_set = np.zeros([n_sam, n_val])
    for i in range(n_val):
        sam_set[:, i] = permutation(bins_low+rand(n_sam)*(bins_high-bins_low))

    # Return sam_set
    return(sam_set)


def _lhd_fixed(n_sam, n_val):
    # Generate the maximally spaced values in every dimension
    val = np.linspace(0, 1, n_sam)

    # Pair values randomly together to obtain random samples
    sam_set = np.zeros([n_sam, n_val])
    for i in range(n_val):
        sam_set[:, i] = permutation(val)

    # Return sam_set
    return(sam_set)


def _lhd_center(n_sam, n_val):
    # Generate the equally spaced intervals/bins
    bins = np.linspace(0, 1, n_sam+1)

    # Obtain lower and upper bounds of bins
    bins_low = bins[0:n_sam]
    bins_high = bins[1:n_sam+1]

    # Capture centers of every bin
    center_num = (bins_low+bins_high)/2

    # Pair values randomly together to obtain random samples
    sam_set = np.zeros([n_sam, n_val])
    for i in range(n_val):
        sam_set[:, i] = permutation(center_num)

    # Return sam_set
    return(sam_set)


time: 2.43 ms


In [40]:
data = lhd(5, 10, method='fixed')

time: 1.88 ms


In [41]:
data

array([[0.5 , 0.  , 0.  , 0.25, 0.25, 1.  , 0.  , 0.75, 0.5 , 0.25],
       [0.25, 0.25, 1.  , 1.  , 1.  , 0.  , 0.5 , 0.25, 1.  , 0.75],
       [0.75, 0.75, 0.75, 0.75, 0.5 , 0.75, 1.  , 0.  , 0.  , 0.  ],
       [1.  , 0.5 , 0.25, 0.5 , 0.75, 0.5 , 0.25, 1.  , 0.25, 1.  ],
       [0.  , 1.  , 0.5 , 0.  , 0.  , 0.25, 0.75, 0.5 , 0.75, 0.5 ]])

time: 2.53 ms


In [38]:
scaler = MinMaxScaler((0, 1e5))
data[:, 0] = scaler.fit_transform(data[:, 0].reshape(-1, 1)).reshape(1, -1)[0]
# data[:, 0]

time: 674 µs


# Artificial bee colony

In [ ]:
# TODO: remove mutation / crossover
# problem dim 16

class ABC:
    def __init__(self, colony_size, problem_dim, init_method='latin_hypercube'):
        self.colony_size = colony_size
        self.problem_dim = problem_dim
        self.scout_limit = (colony_size * problem_dim) / 2
        self.food_resources_num = colony_size / 2
        self.employed_bees = None
        self.init_resources_method = self._init_method(init_method)
        
        # variable limits
        self.low_decor = 0
        self.high_decor = 1e5
        self.low_n = 0
        self.high_n = 15 # changed from 8 to 15
        self.low_sm = 1e-3
        self.high_sm = 1e2
        self.low_sp = -1e2
        self.high_sp = -1e-3ы
        self.low_back = 0
        self.high_back = 8
        self.low_mutation_prob = 0 # TODO: check low and high vals
        self.high_mutation_prob = 1
        self.low_elem_mutation_prob = 0
        self.high_elem_mutation_prob = 1
        self.low_ext_mutation_selector_prob = 0
        self.high_ext_mutation_selector_prob = 1
        
    def init_food_sources_method(self, method):
        if method == 'latin_hypercube':
            return self._latin_hypercube_init_food
        elif method == 'random':
            return self._random_init_food
        
    def _latin_hypercube_init_food(self, 
                                   resources_num=self.food_resources_num):
        params_limits = [
            (self.low_decor, self.high_decor),
            (self.low_n, self.high_n),
            (self.low_sm, self.high_sm),
            (self.low_sm, self.high_sm),
            (self.low_n, self.high_n),
            (self.low_sp, self.high_sp),
            (self.low_sp, self.high_sp),
            (self.low_n, self.high_n),
            (self.low_sp, self.high_sp),
            (self.low_sp, self.high_sp),
            (self.low_n, self.high_n),
            (self.low_back, self.high_back),
            (self.low_mutation_prob, self.high_mutation_prob),
            (self.low_elem_mutation_prob, self.high_elem_mutation_prob),
            (self.low_ext_mutation_selector_prob, self.high_ext_mutation_selector_prob),
            (self.low_decor, self.high_decor)
        ]
        cube_params = lhd(resources_num, self.problem_dim, method='fixed')
        for ix, param_lim in enumerate(params_limits):
            scaler = MinMaxScaler(param_lim)
            cube_params[:, ix] = scaler.fit_transform(cube_params[:, ix].reshape(-1, 1)).reshape(1, -1)[0]
        list_of_individuals = []
        for row in cube_params:
            list_of_individuals.append(IndividualDTO(id=str(uuid.uuid4()), 
                                        params=_int_check(row)))
        self.employed_bees = parallel_fitness(list_of_individuals)
            
        
        
    def _random_init_food(self, 
                          resources_num=self.food_resources_num):
        list_of_individuals = []
        for _ in range(self.food_resources_num):
            val_decor = np.random.uniform(low=self.low_decor, high=self.high_decor, size=1)[0]
            var_n = np.random.randint(low=self.low_n, high=self.high_n, size=5)
            var_sm = np.random.uniform(low=self.low_sm, high=self.high_sm, size=2)
            var_sp = np.random.uniform(low=self.low_sp, high=self.high_sp, size=4)
            var_b = np.random.randint(low=self.low_back, high=self.high_back, size=1)[0]
            ext_mutation_prob = np.random.uniform(low=self.low_mutation_prob, high=self.high_mutation_prob, size=1)[0]
            ext_elem_mutation_prob = np.random.uniform(low=self.low_elem_mutation_prob, high=self.high_elem_mutation_prob, size=1)[0]
            ext_mutation_selector = np.random.uniform(low=self.low_ext_mutation_selector_prob, high=self.high_ext_mutation_selector_prob, size=1)[0]
            val_decor_2 = np.random.uniform(low=self.low_decor, high=self.high_decor, size=1)[0]
            params = [
                    val_decor, var_n[0], 
                    var_sm[0], var_sm[1], var_n[1], 
                    var_sp[0], var_sp[1], var_n[2], 
                    var_sp[2], var_sp[3], var_n[3],
                    var_b,  # TODO: check if it is back
                    ext_mutation_prob, ext_elem_mutation_prob, ext_mutation_selector,
                    val_decor_2
                ]
            params = [float(i) for i in params]
            list_of_individuals.append(IndividualDTO(id=str(uuid.uuid4()), 
                                        params=params))
        self.employed_bees = parallel_fitness(list_of_individuals)
        
    def run(self, iterations):
        # init population
        self.init_resources_method()
        for i in range(iterations):
            # employed bees phase
            new_employed_bees_solutions = []
            for bee in self.employed_bees:
                r = random.random()
                # neighborhood
                change_param = (int)(r * self.problem_dim)
                var_n = np.random.randint(low=0, high=self.food_resources_num, size=1)[0]
                
            pass
            
            
    def _int_check(params):
        res = list(params)
        for i in [1, 4, 7, 10, 11]:
            res[i] = float(np.round(res[i]))
        return res

In [43]:
random.random()

0.2382420914091622

time: 1.45 ms
